In [8]:
!pip install -qU bbox-utility

     |████████████████████████████████| 58 kB 2.6 MB/s 


In [9]:
from bbox.utils import coco2yolo, coco2voc, voc2yolo, voc2coco
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

In [1]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
import torch
from PIL import Image

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12727, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 12727 (delta 45), reused 48 (delta 22), pack-reused 12633
Receiving objects: 100% (12727/12727), 12.95 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (8754/8754), done.


In [4]:
ROOT_DIR  = '/content/drive/MyDrive/hackathon/datasets/Тестовый датасет/IMG'
CKPT_PATH = '/content/drive/MyDrive/hackathon2/web/best.pt'
IMG_SIZE  = 1000
CONF      = 0.15
IOU       = 0.25
AUGMENT   = True

In [5]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/content/yolov5',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

In [17]:
np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [55]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, img_name):
    annots = []

    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            xcen = int(xmin + w/2)
            ycen = int(ymin + h/2)
            out = f'{img_name},{xcen},{ycen}'
            annots.append(out)
    return annots
    

In [ ]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
image_paths = glob.glob(os.path.join("/content/drive/MyDrive/hackathon/datasets/Тестовый датасет/IMG", "*"))
result_path = "/content/drive/MyDrive/hackathon2/МИРЭА.csv"


for path in tqdm(image_paths):
    img = np.array(Image.open(path).convert('RGB'))
    #img = cv2.imread(path)[...,::-1]
    img_name = path.split("/")[-1]

    bboxes, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    outs = format_prediction(bboxes, img_name)
    with open(result_path, 'a') as f:
         for out in outs:
             f.write(out)
             f.write('\n')


YOLOv5 🚀 v6.2-162-g55fbac9 Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 346 layers, 76118664 parameters, 0 gradients
Adding AutoShape... 


  0%|          | 0/500 [00:00<?, ?it/s]

In [61]:
import pandas as pd

df = pd.read_csv(result_path)
df

,name,x,y
